In [ ]:
import torch
from torch import nn

In [ ]:
size = (2,3)
torch.device('cuda')
test = torch.Tensor()
test.new_ones(size)

In [ ]:
input_data = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Mauris commodo nibh at justo ultrices, a consectetur elit venenatis. Integer aliquam nulla nec nisl hendrerit, quis cursus nulla lobortis. Vivamus tincidunt, magna nec consequat imperdiet, urna arcu cursus elit, ultricies porttitor eros urna at velit. Nulla egestas nulla viverra viverra tempor. Pellentesque laoreet risus nisi, ac ultrices ex placerat nec. Vestibulum elementum tellus vel rhoncus vulputate. Pellentesque dictum dui sit amet vestibulum aliquet. Etiam ullamcorper at purus a auctor. Vivamus viverra libero sit amet leo efficitur, ut tincidunt eros placerat. Aenean egestas vitae velit et elementum. Duis fringilla luctus tortor et malesuada. Duis sit amet ipsum semper, condimentum est vel, consequat ante. In hac habitasse platea dictumst. Etiam elementum faucibus mauris. Suspendisse aliquet sapien sit amet elit dignissim posuere. Sed vulputate condimentum libero ac lacinia. Cras vitae fringilla odio, malesuada porttitor est. Quisque at tristique nunc, et volutpat sapien. Suspendisse efficitur, tortor in mollis sagittis, justo metus interdum nisl, sit amet varius sapien lorem eget justo. Aliquam ultrices ligula vehicula, lobortis lacus at, gravida enim. Ut suscipit pretium odio vitae vestibulum. Mauris molestie lobortis dolor vel scelerisque. Vivamus posuere, ligula eget interdum ullamcorper, neque ante accumsan diam, quis sollicitudin lectus diam eu metus. Duis ac arcu fermentum, auctor felis ut, aliquet ligula. Nulla commodo semper euismod. Donec bibendum lorem vitae ornare aliquet. Duis eleifend lectus enim. Morbi risus felis, accumsan a felis non, rhoncus ornare quam. Mauris non augue quis diam tempor aliquam eget aliquet erat. Donec imperdiet dignissim sapien, ut blandit ligula tincidunt sed. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Cras malesuada vitae leo ut dignissim. Donec sed nisl lectus. Vestibulum a luctus elit. Aenean eu aliquam nisi. Phasellus at massa justo. Vivamus tincidunt ante quis felis feugiat, sagittis lacinia sem tempus. Nullam dignissim mi nunc, sit amet dapibus magna aliquam ut. Donec ut risus magna. Praesent augue nisi, lobortis vitae posuere non, mollis at ipsum. Duis malesuada sed enim ac aliquam. Nam orci nulla, lacinia vel ligula eget, tempor semper velit. Nullam aliquam aliquet mattis. Mauris non congue ipsum, vel luctus quam. Vivamus volutpat ligula magna, quis dignissim diam iaculis ac. Fusce commodo feugiat magna ut feugiat. Sed vehicula viverra mauris, et tincidunt ante euismod at. Pellentesque at tortor nec tortor varius sollicitudin."
input_test = "Morbi ac cursus odio, quis mattis felis. Pellentesque gravida sapien congue nibh aliquet aliquet at sed sem. Suspendisse potenti. Praesent iaculis arcu at viverra rutrum. Quisque metus diam, ornare ac ante vitae, auctor rutrum nibh. Interdum et malesuada fames ac ante ipsum primis in faucibus. Nulla gravida risus a lacus cursus luctus. Mauris egestas quis purus scelerisque tempor. Nulla non aliquam mi. Nullam in purus ut metus venenatis condimentum eu dignissim tellus. Nunc maximus interdum erat eu congue. Nullam consectetur lectus sed tincidunt ullamcorper. Nunc ut tellus tincidunt, dapibus mi nec, mattis felis. Donec ut nisl ultrices, facilisis nisl id, pellentesque mi. Quisque varius sodales odio, sed volutpat dolor interdum non."
def work_input(data):
    return [ord(elem) for elem in list(data)]
data = work_input(input_data)
test = work_input(input_test)

#print(test)


In [ ]:
simple = nn.LSTM(1, 10, 1)
simple.train(data)

print(simple(torch.Tensor(test)))
#simple.forward(torch.Tensor(test))